# BrainAge Project

In [23]:
import pandas as pd
import numpy as np
import nibabel as nib
from sklearn.svm import LinearSVR
# from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import VarianceThreshold, SelectKBest, f_regression
import os
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error, r2_score
from nilearn.input_data import NiftiMasker
import random
import matplotlib.pyplot as plt
import datetime

In [20]:
# Config
toy = True
run_all = False

path = '/home/ubuntu/fmriprep/'
# output_dir = '/output/' #AWS cloud
output_dir = 'data/CNP_T1/' #For Daniel's computer

n_jobs = 20 #amount of cores
cv=4

description = 'CNP_T1_validation_age_'
log_file = description+datetime.datetime.now().strftime("%y-%m-%d-%H-%M-%S")

In [62]:
# Load data
loaded = np.load(output_dir+'train_set.npz')
X_train, y_train, random_subj_train = loaded['a'], loaded['b'], loaded['c']

array([[51.60878 , 73.58203 , 83.20948 , ..., 43.260704, 40.342197,
        33.994606],
       [44.85154 , 47.15692 , 46.413765, ..., 40.10278 , 39.20425 ,
        38.602734]], dtype=float32)

In [24]:
random_subj_train

array(['10672', '10940', '10361', '10719', '11112', '10356', '11106',
       '11030', '10340', '10570', '10329', '10871', '10746', '10325',
       '10987', '10724', '11105', '10492', '10429', '10565', '10977',
       '10316', '10460', '10678', '10704', '11143', '11067', '10274',
       '10304', '10696', '11098', '10998', '10455', '10629', '10963',
       '10345', '11019', '10271', '11108', '10225', '11149', '11050',
       '10448', '10680', '11068', '11104', '10388', '10530', '10206',
       '10692', '10438', '10228', '10762', '10708', '10235', '10788',
       '10527', '10631', '11044', '10844', '10517', '11090', '10377',
       '10934', '10575', '10471', '10159', '10189', '10779', '10949',
       '11052', '11059', '10280', '11156', '11077', '10975', '10290',
       '10557', '11066', '10686', '10891', '10525', '11061', '10321'],
      dtype='<U5')

In [64]:
# loaded = np.load(output_dir+'test_set.npz')
# random_subj_test = loaded['c']
# random_subj_test

# subj_order = np.concatenate([random_subj_train, random_subj_test])
# subj_order

# np.save('final_subj_order', subj_order)


In [21]:
print(y_train.shape)
print(X_train.shape)

(84,)
(84, 1899247)


In [22]:
X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=0.30, random_state=42)

## Validation 

Grid search returned the best parameters:

* ANOVA k selection: 5000
* C parameter of SVR: 0.01


In [69]:
from sklearn.pipeline import Pipeline

toy=True
if toy:
    X_train = X_train[:8]
    y_train = y_train[:8]
    X_validation = X_validation[:8]
    y_validation = y_validation[:8]


# Remove features with too low between-subject variance
# Here we use a classical univariate feature selection based on F-test,
# namely Anova.
# We have our predictor (SVR), our feature selection (SelectKBest), and now,
# we can plug them together in a *pipeline* that performs the two operations
# successively:
               
anova_svr = Pipeline([
            ('anova', SelectKBest(f_regression, k=5000)),
            ('svr', SVR(kernel='linear', C=0.01))])

# parameters = [{'anova__k': [2000],
#                'svr__C': [0.1,1]}]


anova_svr.fit(X_train, y_train)
predictions = anova_svr.predict(X_validation)

# clf = SVR(kernel='linear', C=0.01)
# clf.fit(X_train, y_train)
# predictions = clf.predict(X_validation)



score = mean_absolute_error(y_validation, predictions)
r2 = r2_score(y_validation, predictions)
# mean_scores = np.array(grid.cv_results_['neg_mean_absolute_error'])
# mean_scores = mean_scores.reshape(len(C_OPTIONS), -1, len(N_FEATURES_OPTIONS))
# # select score for best C
# mean_scores = mean_scores.max(axis=0)
# bar_offsets = (np.arange(len(N_FEATURES_OPTIONS)) *
#                (len(reducer_labels) + 1) + .5)






In [70]:
print(score)
print(predictions)
print(y_validation)

5.516526634116861
[28.28407341 28.8998851  35.5416655  28.90332036 27.42821537 29.41925641
 30.0392139  45.30632493]
[25 29 40 40 23 26 22 36]


In [32]:
print('best score: ',grid.best_score_)
print('best params: ',grid.best_params_)
print('\n\nfullresults: \n', grid.cv_results_)
df1 = pd.DataFrame(grid.cv_results_)
df1

NameError: name 'grid' is not defined

In [ ]:
age_pred = grid.predict(X_train)
mean_abs_error = np.mean(np.abs(age_pred-y_train))
df1 = pd.DataFrame(grid.cv_results_)

with open(output_dir+'log_'+log_file+'.txt', '+a') as f:
    f.write('mean absolute error: '+str(score.round(4))+'\n')
    f.write('r2: '+str(r2.round(4))+'\n')
    f.write(str(predictions)+'\n')
#     f.write('\n\nfullresults: \n'+str(grid.cv_results_)+'\n')
#     f.write(str(df1)+'\n')
#     f.write(str(age_pred)+'\n')
#     f.write(str(y_train)+'\n')
#     f.write(str(mean_abs_error)+'\n')
    

In [ ]:
# age_pred = grid.predict(gm_maps_masked)
# np.mean(np.abs(age_pred-y_age))
# y_age

In [ ]:
# r2 score